### Extract Local Film Data

In [ ]:
from pathlib import Path
import csv
import sys

# --------------- Ruta Modificable Aquí --------------
root = Path(r"E:\VIDEOS\FILME")
out_path = Path("filme_report.csv")
# ----------------------------------------------------

VIDEO_EXTS = {
    '.mp4', '.mkv', '.avi', '.mov', '.wmv', '.flv', '.webm',
    '.mpg', '.mpeg', '.m4v', '.3gp', '.ts', '.rmvb'
}


def human_readable_size(nbytes: int) -> str:
    if nbytes < 1024:
        return f"{nbytes} B"
    for unit in ("KB", "MB", "GB", "TB"):
        nbytes /= 1024.0
        if nbytes < 1024.0:
            return f"{nbytes:3.1f} {unit}"
    return f"{nbytes:.1f} PB"


def get_folder_size(folder: Path) -> int:
    """Suma recursivamente el tamaño de todos los archivos dentro de una carpeta."""
    total = 0
    for f in folder.rglob("*"):
        if f.is_file():
            try:
                total += f.stat().st_size
            except OSError:
                pass
    return total


def examine_films(root: Path):
    results = []
    folders_with_subfolders = 0

    if not root.exists() or not root.is_dir():
        raise FileNotFoundError(f"La carpeta raíz indicada no existe o no es un directorio: {root}")

    for entry in sorted(root.iterdir(), key=lambda p: p.name.lower()):
        if not entry.is_dir():
            continue

        carpeta = entry
        archivos = [f for f in carpeta.iterdir() if f.is_file()]
        subcarpetas = [d for d in carpeta.iterdir() if d.is_dir()]

        num_docs = len(archivos)
        nombres_docs = [f.name for f in archivos]

        # Detectar archivos de vídeo por extensión
        video_files = [f for f in archivos if f.suffix.lower() in VIDEO_EXTS]
        num_videos = len(video_files)
        nombres_videos = [f.name for f in video_files]

        # Tamaños (bytes y legible)
        tamaños_bytes = []
        tamaños_legibles = []
        formatos = []
        for f in video_files:
            try:
                size = f.stat().st_size
            except OSError:
                size = 0
            tamaños_bytes.append(str(size))
            tamaños_legibles.append(human_readable_size(size))
            formatos.append(f.suffix.lower().lstrip('.'))

        # Detectar carpeta VIDEO_TS y calcular su tamaño total
        has_videots = None
        for d in subcarpetas:
            if d.name.lower() == "video_ts":
                has_videots = d
                break

        tam_videots_bytes = ""
        tam_videots_legible = ""
        if has_videots:
            formato_field = "DVD"
            size_videots = get_folder_size(has_videots)
            tam_videots_bytes = str(size_videots)
            tam_videots_legible = human_readable_size(size_videots)
        else:
            # Unir formatos únicos en orden alfabético
            unique_formats = sorted(set(formatos))
            formato_field = "; ".join(unique_formats) if unique_formats else ""

        num_subcarpetas = len(subcarpetas)
        nombres_subcarpetas = [d.name for d in subcarpetas]

        if num_subcarpetas > 0:
            folders_with_subfolders += 1

        results.append({
            "CARPETA": carpeta.name,
            "Numero de documentos": num_docs,
            "Nombres de documentos": "; ".join(nombres_docs),
            "Número de documentos de tipo video": num_videos,
            "Nombre de documentos de tipo video": "; ".join(nombres_videos),
            "Tamaño (bytes)": "; ".join(tamaños_bytes),
            "Tamaño (legible)": "; ".join(tamaños_legibles),
            "Formato de video": formato_field,
            "Tamaño VIDEO_TS (bytes)": tam_videots_bytes,
            "Tamaño VIDEO_TS (legible)": tam_videots_legible,
            "Numero de subcarpetas": num_subcarpetas,
            "Nombre de subcarpetas": "; ".join(nombres_subcarpetas),
        })

    return results, folders_with_subfolders


def write_csv(rows, out_path: Path, delimiter="|"):
    headers = [
        "CARPETA",
        "Numero de documentos",
        "Nombres de documentos",
        "Número de documentos de tipo video",
        "Nombre de documentos de tipo video",
        "Tamaño (bytes)",
        "Tamaño (legible)",
        "Formato de video",
        "Tamaño VIDEO_TS (bytes)",
        "Tamaño VIDEO_TS (legible)",
        "Numero de subcarpetas",
        "Nombre de subcarpetas",
    ]
    with out_path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=headers, delimiter=delimiter)
        writer.writeheader()
        writer.writerows(rows)


if __name__ == "__main__":
    try:
        rows, folders_with_subfolders = examine_films(root)
        write_csv(rows, out_path)
        print(f"\n✅ CSV generado en: {out_path.resolve()}")
        print(f"📁 Carpetas con subcarpetas: {folders_with_subfolders}")
    except Exception as e:
        print("❌ Error:", e)

### Identify Local Films to Enrich Film Data

In [ ]:
from pathlib import Path
import requests
import time
import csv
import re
from bs4 import BeautifulSoup
from urllib.parse import quote_plus

# ---------------- CONFIG ----------------
root = Path(r"E:\VIDEOS\FILME")
out_csv = Path("filme_metadata.csv")
OMDB_API_KEY = "http://www.omdbapi.com/?i=tt3896198&apikey=c53c64d6"
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "\
             "(KHTML, like Gecko) Chrome/117.0 Safari/537.36"
REQUEST_DELAY = 3.0
# ----------------------------------------

session = requests.Session()
session.headers.update({"User-Agent": USER_AGENT})


def safe_get(url, params=None, timeout=15):
    """Petición GET con manejo básico de errores y delays."""
    try:
        r = session.get(url, params=params, timeout=timeout)
        time.sleep(REQUEST_DELAY)
        r.raise_for_status()
        return r
    except Exception as e:
        # print(f"[WARN] petición fallida a {url}: {e}")
        return None


def search_omdb(title, year=None):
    """
    Busca en OMDb por título (y año opcional) y devuelve el JSON si lo encuentra.
    """
    if not OMDB_API_KEY or OMDB_API_KEY == "TU_OMDB_API_KEY_AQUI":
        return None
    params = {"apikey": OMDB_API_KEY, "t": title}
    if year:
        params["y"] = year
    r = safe_get("http://www.omdbapi.com/", params=params)
    if not r:
        return None
    data = r.json()
    if data.get("Response", "False") == "True":
        return data
    # fallback: intentar búsqueda por s (search) y luego obtener por imdbID
    params_search = {"apikey": OMDB_API_KEY, "s": title}
    r2 = safe_get("http://www.omdbapi.com/", params=params_search)
    if not r2:
        return None
    sr = r2.json()
    if sr.get("Response") == "True" and sr.get("Search"):
        first = sr["Search"][0]
        imdbid = first.get("imdbID")
        r3 = safe_get("http://www.omdbapi.com/", params={"apikey": OMDB_API_KEY, "i": imdbid, "plot": "short"})
        if r3:
            d = r3.json()
            if d.get("Response") == "True":
                return d
    return None


def parse_awards_for_oscars(awards_text):
    """
    Intenta extraer nominaciones y victorias de los Oscars y edición si aparece.
    Devuelve (nominaciones, ganados, edicion_texto)
    """
    if not awards_text:
        return "", "", ""
    text = awards_text
    # buscar 'Oscar' seguido de números
    nominaciones = ""
    ganados = ""
    edicion = ""
    # ejemplos en OMDb: "Nominated for 1 Oscar. Another 3 wins & 5 nominations."
    # buscar "Nominated for X Oscar" o "Won X Oscar"
    m_nom = re.search(r"Nominated for (\d+) Oscar", text)
    m_won = re.search(r"Won (\d+) Oscar", text)
    if m_nom:
        nominaciones = m_nom.group(1)
    if m_won:
        ganados = m_won.group(1)
    # a veces aparece "Nominated for 2 Oscars. Another 3 wins & 4 nominations."
    # También intentar buscar "Oscars" plural
    if not nominaciones:
        m_nom2 = re.search(r"Nominated for (\d+) Oscars", text)
        if m_nom2:
            nominaciones = m_nom2.group(1)
    if not ganados:
        m_won2 = re.search(r"Won (\d+) Oscars", text)
        if m_won2:
            ganados = m_won2.group(1)
    # edición: intentar capturar "Nominated for X Oscar (YEAR?)" no siempre está
    # No hay formato estándar; devolvemos texto completo si contiene 'Oscar'
    if "Oscar" in text:
        edicion = text
    return nominaciones, ganados, edicion


def search_filmaffinity(folder_name):
    """
    Buscar en FilmAffinity el título y devolver (fa_id, fa_rating, fa_synopsis).
    Método: usar el buscador de FilmAffinity y tomar el primer resultado.
    NOTA: FilmAffinity puede cambiar HTML o bloquear scraping.
    """
    try:
        query = quote_plus(folder_name)
        url = f"https://www.filmaffinity.com/en/search.php?stype=title&stext={query}"
        # en ocasiones el dominio /es/ o sin /en/; probar /en/ para consistencia
        r = safe_get(url)
        if not r:
            # intentar versión sin /en/
            url2 = f"https://www.filmaffinity.com/search.php?stype=title&stext={query}"
            r = safe_get(url2)
            if not r:
                return "", "", ""
        soup = BeautifulSoup(r.text, "html.parser")
        # buscar primer resultado en la lista
        # en la página: resultados suelen estar en .movie-card / .fa-search-result / .mc-title
        link = soup.select_one("div.movie-card a[href]") or soup.select_one("div.mc-title a[href]") or soup.select_one("a.movie-link[href]")
        if not link:
            # intentar otro selector general
            link = soup.select_one("a[href*='film']")  # enlace que contenga 'film'
        if not link:
            return "", "", ""
        href = link.get("href")
        # obtener id si está en la URL, ej: /en/film123456.html o /film123456.html
        m = re.search(r"film(\d+)\.html", href)
        fa_id = m.group(1) if m else href
        # solicitar la página del film
        film_url = href if href.startswith("http") else ("https://www.filmaffinity.com" + href)
        r2 = safe_get(film_url)
        if not r2:
            return fa_id, "", ""
        s2 = BeautifulSoup(r2.text, "html.parser")
        # rating: puede estar en el selector .rating .average or span[itemprop="ratingValue"]
        rating_el = s2.select_one('div.avg-rating') or s2.select_one('div.rating') or s2.select_one('span[itemprop="ratingValue"]')
        rating = ""
        if rating_el:
            rating = rating_el.get_text(strip=True)
        # sinopsis: buscar #synopsis or .synopsis or #sinopsis
        sinopsis_el = s2.select_one("#synopsis") or s2.select_one("div.synopsis") or s2.select_one("div#movie-synopsis")
        sinopsis = ""
        if sinopsis_el:
            sinopsis = sinopsis_el.get_text(" ", strip=True)
        # versión en español: FilmAffinity muestra título original y título español en la misma página; extraer título original si hay
        return fa_id, rating, sinopsis
    except Exception:
        return "", "", ""


def search_rottentomatoes(folder_name):
    """
    Busca en RottenTomatoes y devuelve (rt_id, tomatometer, popcornmeter?).
    Usamos la búsqueda pública: https://www.rottentomatoes.com/search?search=...
    """
    try:
        q = quote_plus(folder_name)
        url = f"https://www.rottentomatoes.com/search?search={q}"
        r = safe_get(url)
        if not r:
            return "", "", ""
        soup = BeautifulSoup(r.text, "html.parser")
        # La estructura JS a veces rellena resultados; intentar leer resultados "search-page-media-row" o links a /m/slug
        link = soup.select_one("search-page-media-row a") or soup.select_one("a[href^='/m/']") or soup.select_one("a[href^='/m/'], a[href^='/movies/']")
        if not link:
            return "", "", ""
        href = link.get("href")
        rt_id = href
        film_url = href if href.startswith("http") else ("https://www.rottentomatoes.com" + href)
        r2 = safe_get(film_url)
        if not r2:
            return rt_id, "", ""
        s2 = BeautifulSoup(r2.text, "html.parser")
        # Tomatometer: selector puede ser score-board with data-meter or scoreBoard
        tm = ""
        # Buscamos elementos con class 'mop-ratings-wrap__percentage' o 'score-board' data-meter
        score_el = s2.select_one(".mop-ratings-wrap__percentage") or s2.select_one("score-board")
        if score_el:
            tm_text = score_el.get_text(" ", strip=True)
            m = re.search(r"(\d+)%", tm_text)
            if m:
                tm = m.group(1)
        # si score-board tiene atributo data-meter
        sb = s2.select_one("score-board")
        if sb and not tm:
            tm = sb.get("tomatometerscore") or sb.get("tomatometerscore")
        return rt_id, tm, ""
    except Exception:
        return "", "", ""


def search_popcornmeter(folder_name):
    """
    Intento simple para PopcornMeter (si existe); devolver el id o puntuación si se encuentra.
    """
    try:
        q = quote_plus(folder_name)
        url = f"https://www.popcornmeter.com/search?search={q}"
        r = safe_get(url)
        if not r:
            return "", ""
        soup = BeautifulSoup(r.text, "html.parser")
        link = soup.select_one("a[href*='/movie/']")
        if not link:
            return "", ""
        href = link.get("href")
        # la puntuación puede aparecer en .popcornmeter-score o similar
        r2 = safe_get(href if href.startswith("http") else ("https://www.popcornmeter.com" + href))
        if not r2:
            return href, ""
        s2 = BeautifulSoup(r2.text, "html.parser")
        score_el = s2.select_one(".score") or s2.select_one(".rating-value")
        score = score_el.get_text(strip=True) if score_el else ""
        return href, score
    except Exception:
        return "", ""


def collect_for_folder(folder: Path):
    """
    Dada la carpeta de la película, retorna un dict con los campos solicitados.
    """
    folder_name = folder.name
    row = {
        "Título original": "",
        "Título en español": "",
        "Dirección": "",
        "Guión": "",
        "País": "",
        "año": "",
        "Duración": "",
        "Lista de Géneros": "",
        "Puntuación filmaffinity": "",
        "Puntuación IMDB": "",
        "Tomatometer": "",
        "Popcornmeter": "",
        "Lista de actores": "",
        "Nominaciones Oscar": "",
        "Oscar ganados": "",
        "Edición Oscar": "",
        "Sinopsis filmaffinity": "",
        "ID OMDb": "",
        "ID IMDB": "",
        "ID filmaffinity": "",
        "ID rottentomatoes": "",
        "Nombre de la carpeta": folder_name,
    }

    # 1) Intentar OMDb (clave obligatoria)
    omdb_data = search_omdb(folder_name)
    if omdb_data:
        # rellenar campos desde OMDb
        row["Título original"] = omdb_data.get("Title", "")
        # OMDb no distingue título en español; dejar Título en español vacío si no hay
        row["Título en español"] = ""  # opcional: podríamos intentar traducir o buscar en FilmAffinity
        row["Dirección"] = omdb_data.get("Director", "")
        row["Guión"] = omdb_data.get("Writer", "")
        row["País"] = omdb_data.get("Country", "")
        row["año"] = omdb_data.get("Year", "")
        row["Duración"] = omdb_data.get("Runtime", "")
        row["Lista de Géneros"] = omdb_data.get("Genre", "")
        # IMDb rating
        row["Puntuación IMDB"] = omdb_data.get("imdbRating", "")
        row["Lista de actores"] = omdb_data.get("Actors", "")
        # IMDb ID
        row["ID IMDB"] = omdb_data.get("imdbID", "")
        # OMDb ID: no tiene id propio, usaremos imdbID como referencia
        row["ID OMDb"] = row["ID IMDB"]
        # Awards (para Oscars)
        nomin, gan, ed = parse_awards_for_oscars(omdb_data.get("Awards", ""))
        row["Nominaciones Oscar"] = nomin
        row["Oscar ganados"] = gan
        row["Edición Oscar"] = ed

    # 2) FilmAffinity (scrape)
    fa_id, fa_rating, fa_synopsis = search_filmaffinity(folder_name)
    if fa_id:
        row["ID filmaffinity"] = fa_id
    if fa_rating:
        row["Puntuación filmaffinity"] = fa_rating
    if fa_synopsis:
        row["Sinopsis filmaffinity"] = fa_synopsis

    # 3) RottenTomatoes
    rt_id, tomatometer, _ = search_rottentomatoes(folder_name)
    if rt_id:
        row["ID rottentomatoes"] = rt_id
    if tomatometer:
        row["Tomatometer"] = tomatometer

    # 4) PopcornMeter
    pcm_id, pcm_score = search_popcornmeter(folder_name)
    if pcm_score:
        row["Popcornmeter"] = pcm_score

    return row


def main():
    if not root.exists() or not root.is_dir():
        print(f"[ERROR] La ruta root no existe: {root}")
        return

    headers = [
        "Título original", "Título en español", "Dirección", "Guión", "País", "año",
        "Duración", "Lista de Géneros", "Puntuación filmaffinity", "Puntuación IMDB",
        "Tomatometer", "Popcornmeter", "Lista de actores", "Nominaciones Oscar",
        "Oscar ganados", "Edición Oscar", "Sinopsis filmaffinity",
        "ID OMDb", "ID IMDB", "ID filmaffinity", "ID rottentomatoes", "Nombre de la carpeta"
    ]

    rows = []
    for entry in sorted(root.iterdir(), key=lambda p: p.name.lower()):
        if not entry.is_dir():
            continue
        try:
            print(f"Procesando: {entry.name}")
            r = collect_for_folder(entry)
            rows.append(r)
        except Exception as e:
            print(f"[WARN] fallo procesando {entry.name}: {e}")

    # escribir CSV
    with out_csv.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=headers, delimiter="|")
        writer.writeheader()
        for r in rows:
            writer.writerow(r)

    print(f"CSV generado: {out_csv.resolve()}")


if __name__ == "__main__":
    main()
